# PostScript


In [ ]:
var fso = new ActiveXObject("Scripting.FileSystemObject");

// 打開PostScript檔案
function openPSFile(filePath) {
    if (!fso.FileExists(filePath)) {
        WScript.Echo("File not found: " + filePath);
        return null;
    }
    var file = fso.OpenTextFile(filePath, 1);
    return file;
}

// 讀取PostScript檔案的所有內容
function readPSFile(file) {
    var content = "";
    while (!file.AtEndOfStream) {
        content += file.ReadLine() + "\n";
    }
    return content;
}

// 抓取特定頁面標籤區
function getPageSections(content) {
    var pages = content.split("%%Page: ");
    var pageSections = [];
    for (var i = 1; i < pages.length; i++) {
        pageSections.push("%%Page: " + pages[i]);
    }
    return pageSections;
}

// 創建指令物件
function createCommand(commandLines) {
    return commandLines.join("\n");
}

// 將指令物件推送到特定頁面標籤區
function pushCommandToPageSection(pageSection, command) {
    var setupIndex = pageSection.indexOf("%%BeginPageSetup");
    if (setupIndex === -1) {
        WScript.Echo("Page setup section not found");
        return pageSection;
    }
    var endSetupIndex = pageSection.indexOf("%%EndPageSetup", setupIndex);
    if (endSetupIndex === -1) {
        WScript.Echo("Page setup end section not found");
        return pageSection;
    }
    var beforeSetup = pageSection.substring(0, setupIndex + 16); // "%%BeginPageSetup" length is 16
    var afterSetup = pageSection.substring(endSetupIndex);
    return beforeSetup + "\n" + command + "\n" + afterSetup;
}

// 保存並關閉檔案
function saveAndCloseFile(filePath, content) {
    var file = fso.OpenTextFile(filePath, 2, true);
    file.Write(content);
    file.Close();
}

// 主函數
function PSPageCommand(filePath, pageIndex, commandLines) {
    var file = openPSFile(filePath);
    if (file === null) return;

    var content = readPSFile(file);
    file.Close();

    var pageSections = getPageSections(content);

    if (pageIndex < 1 || pageIndex > pageSections.length) {
        WScript.Echo("Page index out of range");
        return;
    }

    var command = createCommand(commandLines);
    pageSections[pageIndex - 1] = pushCommandToPageSection(pageSections[pageIndex - 1], command);

    var newContent = pageSections.join("\n");
    saveAndCloseFile(filePath, newContent);
}

// 使用範例
var filePath = "path\\to\\your\\file.ps";
var pageIndex = 1; // 指定頁面索引
var commandLines = ["<< /Staple 3 >> setpagedevice"];

PSPageCommand(filePath, pageIndex, commandLines);
